# Understanding FEM with Ferrite

## Assemble() function

Link with `assemble!()` function from Ferrite and FEM courses:

**FEM in brief**
Solving **Partial Differential Equations (PDEs)** by transforming them into a system of algebraic equations.

*1. Discretization of the domain*
The continuous problem domain is divided into a finite number of sub-domains called **finite elements** (triangles, quadrilaterals in 2D; tetrahedra, hexahedra in 3D).

*2. Interpolation on elements*
The unknown solution ($u$) is approximated on each element by a linear combination of **shape functions** (also called basis functions or interpolation functions) and **nodal values** (Degrees of Freedom, **DoF**) of the solution.
For example, for an element, $u^h(x) \approx \sum_{j=1}^{n} u_j \phi_j(x)$, where $u_j$ are the nodal values and $\phi_j(x)$ are the shape functions.

*3. Variational (weak) formulation*
The PDE is transformed into an integral form (the "weak form"), which is then applied to each element. This leads to a small system of equations for each element, often called the **elemental stiffness matrix** ($K_e$) and **elemental force vector** ($f_e$).

**The Role of Assembly**
Calculate $K_e$ and $f_e$ for each element, then combine them to form the **global stiffness matrix** ($K$) and the **global force vector** ($F$).

* **Identify the contribution of each local DoF to a global DoF**. Shape functions of different elements can overlap at shared nodes.
* **Sum the contributions of each element** to the appropriate position in the global matrix. If a global DoF is affected by multiple elements, their local contributions are simply added.

In Ferrite.jl, the `assemble_element!` and `assemble!` functions precisely manage this:

* `assemble_element!` (which you write): This is where you define the integrals of your weak form for a single element. You calculate the $K_e$ and $f_e$ terms using `CellValues` (for shape functions, gradients, quadrature points, etc.).
* `assemble!` (provided by Ferrite.jl): This function takes the local matrices/vectors ($K_e$, $f_e$) you've calculated and adds them to the correct positions in the global matrices/vectors ($K$, $F$). It handles the "connectivity" and the summation of overlapping contributions. Ferrite does this in an optimized way for sparse matrices.

**Useful Source**
[https://github.com/Ferrite-FEM/Ferrite.jl/blob/f1d1d0deef7bdaf019bd63ce9e8d959b6ebc8c4d/src/assembler.jl#L240-L249](https://github.com/Ferrite-FEM/Ferrite.jl/blob/f1d1d0deef7bdaf019bd63ce9e8d959b6ebc8c4d/src/assembler.jl#L240-L249)

## Exemples

This code visually demonstrates the **FEM assembly process** using the Ferrite.jl library for a simple 1D Poisson problem. It specifically highlights the workflow from mesh generation and local element computations to the efficient construction of the global system matrix using Ferrite's assemble! function.

In [7]:
using Ferrite, SparseArrays

# --- 1. Grid and CellValues (Minimal 1D setup) ---
nels = (10,) # 10 elements
left = Vec((0.0,))
right = Vec((1.0,))
grid = generate_grid(Line, nels, left, right)

ip = Lagrange{RefLine, 1}() # Linear interpolation
qr = QuadratureRule{RefLine}(1) # 1 quadrature point for simplicity
cellvalues = CellValues(qr, ip)

dh = DofHandler(grid)
add!(dh, :u, ip)
close!(dh)

# --- 2. Local Assembly Function ---
# This is where you define how to compute Ke and fe for a single element
function assemble_element!(Ke::Matrix{Float64}, fe::Vector{Float64}, cellvalues::CellValues)
    n_basefuncs = getnbasefunctions(cellvalues)
    fill!(Ke, 0.0) # Clear local stiffness matrix
    fill!(fe, 0.0) # Clear local force vector

    # Loop over quadrature points
    for qp in 1:getnquadpoints(cellvalues)
        dΩ = getdetJdV(cellvalues, qp) # Volume element (length in 1D)
        
        # For -u'' = 0 (homogeneous Poisson), f_val = 0
        # If f = 1, then f_val = 1.0
        f_val = 0.0 

        # Loop over basis functions for the stiffness matrix
        for i in 1:n_basefuncs
            ∇ϕ_i = shape_gradient(cellvalues, qp, i) # Gradient of shape function i
            for j in 1:n_basefuncs
                ∇ϕ_j = shape_gradient(cellvalues, qp, j) # Gradient of shape function j
                Ke[i, j] += (∇ϕ_i ⋅ ∇ϕ_j) * dΩ # (∇ϕ_i ⋅ ∇ϕ_j) is just ∇ϕ_i[1] * ∇ϕ_j[1] in 1D
            end
        end
        
        # Loop over basis functions for the force vector
        for i in 1:n_basefuncs
            ϕ_i = shape_value(cellvalues, qp, i) # Value of shape function i
            fe[i] += ϕ_i * f_val * dΩ
        end
    end
end

# --- 3. Global Assembly Function (uses assemble! internally) ---
function assemble_global_Kf(cellvalues::CellValues, dh::DofHandler)
    K_global = allocate_matrix(dh) # CORRECT: Allocates a sparse matrix with the right sparsity pattern

    f_global = zeros(ndofs(dh)) # Global force vector (this is fine as it's a dense vector)

    # Allocate local buffers
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)

    # Initialize the assembler
    assembler = start_assemble(K_global, f_global)

    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues, cell)
        assemble_element!(Ke, fe, cellvalues)
        cell_dofs = celldofs(cell)
        assemble!(assembler, cell_dofs, Ke, fe)
    end
    
    return K_global, f_global
end

# --- 4. Perform Global Assembly and Display ---
println("Performing global assembly using assemble!...")
K, f = assemble_global_Kf(cellvalues, dh)

println("\nGlobal Stiffness Matrix (K):")
display(K)
println("-"^30)

println("Global Force Vector (f):")
display(f)
println("-"^30)

# --- 5. Interpretation ---
# For a 1D Poisson problem (-u'' = 0) with linear elements,
# Scaled by 1/h (h=0.1 for 10 elements on [0,1], so 1/0.1 = 10)
# So for 10 elements, you'd expect a sparse matrix with entries like:
# diag(K) = [10, 20, 20, ..., 20, 10]
# sub/super-diag(K) = [-10, -10, ..., -10]

Performing global assembly using assemble!...

Global Stiffness Matrix (K):


11×11 SparseMatrixCSC{Float64, Int64} with 31 stored entries:
  10.0  -10.0     ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅ 
 -10.0   20.0  -10.0     ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅ 
    ⋅   -10.0   20.0  -10.0     ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅ 
    ⋅      ⋅   -10.0   20.0  -10.0     ⋅      ⋅      ⋅      ⋅      ⋅      ⋅ 
    ⋅      ⋅      ⋅   -10.0   20.0  -10.0     ⋅      ⋅      ⋅      ⋅      ⋅ 
    ⋅      ⋅      ⋅      ⋅   -10.0   20.0  -10.0     ⋅      ⋅      ⋅      ⋅ 
    ⋅      ⋅      ⋅      ⋅      ⋅   -10.0   20.0  -10.0     ⋅      ⋅      ⋅ 
    ⋅      ⋅      ⋅      ⋅      ⋅      ⋅   -10.0   20.0  -10.0     ⋅      ⋅ 
    ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅   -10.0   20.0  -10.0     ⋅ 
    ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅   -10.0   20.0  -10.0
    ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅   -10.0   10.0

------------------------------
Global Force Vector (f):


11-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

------------------------------


This code conceptualizes how local stiffness contributions from individual elements in a simple 1D mesh (two segments, three nodes) are systematically combined into a larger, global stiffness matrix. The core illustration is the summation of overlapping contributions at shared nodes, clearly showing how element-level properties are integrated to form the system-wide equations. 

In [1]:
# --- 1. Définition du problème simple ---
# On a 3 nœuds (1, 2, 3) et 2 éléments (e1, e2)
# Élément 1 connecte les nœuds 1 et 2
# Élément 2 connecte les nœuds 2 et 3

num_global_dofs = 3 # Nombre de degrés de liberté globaux (ici, les 3 nœuds)

# --- 2. Matrices de rigidité élémentaires (conceptuelles) ---
# En 1D, pour un problème simple (ex: -u'' = f), une matrice élémentaire
# typique pour un élément à 2 nœuds serait:
#       [  1  -1 ]
#       [ -1   1 ]
# Multipliée par une constante (ex: 1/h, où h est la taille de l'élément)

Ke1 = [ 1.0  -1.0 ;
       -1.0   1.0 ] # Matrice de rigidité pour l'élément 1

Ke2 = [ 1.0  -1.0 ;
       -1.0   1.0 ] # Matrice de rigidité pour l'élément 2

# --- 3. Table de connectivité ---
# Indique quels DoF globaux sont connectés par chaque élément.
# Pour e1 (nœuds 1 et 2): ses DoF locaux 1 et 2 correspondent aux DoF globaux 1 et 2.
# Pour e2 (nœuds 2 et 3): ses DoF locaux 1 et 2 correspondent aux DoF globaux 2 et 3.
# C'est l'équivalent de `celldofs(cell)` dans Ferrite.jl.

dofs_e1 = [1, 2] # DoF globaux de l'élément 1
dofs_e2 = [2, 3] # DoF globaux de l'élément 2

# --- 4. Fonction d'assemblage conceptuelle ---
# Cette fonction prend une matrice globale et une matrice élémentaire,
# et ajoute la contribution de la matrice élémentaire aux bons emplacements.

function conceptual_assemble!(K_global::SparseMatrixCSC, Ke_local::Matrix{Float64}, global_dofs::Vector{Int})
    n_local = size(Ke_local, 1) # Nombre de DoF locaux de l'élément
    
    # Vérification simple pour s'assurer que les dimensions correspondent
    if n_local != length(global_dofs)
        error("Mismatch between local matrix size and number of global DoFs provided.")
    end

    # Boucle sur les DoF locaux de l'élément
    for i_local in 1:n_local
        i_global = global_dofs[i_local] # Convertit l'indice local en indice global

        for j_local in 1:n_local
            j_global = global_dofs[j_local] # Convertit l'indice local en indice global
            
            # Additionne la contribution de l'élément à la matrice globale
            K_global[i_global, j_global] += Ke_local[i_local, j_local]
        end
    end
end

# --- 5. Initialisation de la matrice globale ---
# On utilise une matrice creuse comme K_prototype dans Ferrite.jl.
K_global = sparse(Float64[], Int[], Float64[], num_global_dofs, num_global_dofs)

# --- 6. Exécution de l'assemblage pour chaque élément ---

println("Matrice globale initiale (vide):")
display(K_global)
println("-"^30)

println("Assemblage de l'élément 1 (connecte 1 et 2)...")
conceptual_assemble!(K_global, Ke1, dofs_e1)
display(K_global)
println("-"^30)

println("Assemblage de l'élément 2 (connecte 2 et 3)...")
conceptual_assemble!(K_global, Ke2, dofs_e2)
display(K_global)
println("-"^30)

println("Matrice globale finale après assemblage:")
display(K_global)
println("-"^30)

# --- 7. Interprétation du résultat ---
# La matrice finale devrait ressembler à:
# [ 1.0  -1.0   0.0 ]
# [-1.0   2.0  -1.0 ]  
# [ 0.0  -1.0   1.0 ]

Matrice globale initiale (vide):


3×3 SparseMatrixCSC{Float64, Int64} with 0 stored entries:
  ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅ 

------------------------------
Assemblage de l'élément 1 (connecte 1 et 2)...


3×3 SparseMatrixCSC{Float64, Int64} with 4 stored entries:
  1.0  -1.0   ⋅ 
 -1.0   1.0   ⋅ 
   ⋅     ⋅    ⋅ 

------------------------------
Assemblage de l'élément 2 (connecte 2 et 3)...


3×3 SparseMatrixCSC{Float64, Int64} with 7 stored entries:
  1.0  -1.0    ⋅ 
 -1.0   2.0  -1.0
   ⋅   -1.0   1.0

------------------------------
Matrice globale finale après assemblage:


3×3 SparseMatrixCSC{Float64, Int64} with 7 stored entries:
  1.0  -1.0    ⋅ 
 -1.0   2.0  -1.0
   ⋅   -1.0   1.0

------------------------------
